In [1]:
import networkx as nx
import random

class SimulacaoIncendio:
    def __init__(self, num_vertices, num_arestas, postos_brigadistas, pontos_agua, capacidade_caminhoes):
        self.grafo = nx.Graph()
        self.capacidade_caminhoes = capacidade_caminhoes
        
        # Criando os vértices e arestas
        for i in range(num_vertices):
            self.grafo.add_node(i, fogo=False, brigadistas=False, agua=False)
        
        for _ in range(num_arestas):
            u, v = random.sample(range(num_vertices), 2)
            peso = random.randint(1, 10)
            self.grafo.add_edge(u, v, weight=peso)
        
        # Definindo os postos de brigadistas e pontos de água
        for p in postos_brigadistas:
            self.grafo.nodes[p]['brigadistas'] = True
        for p in pontos_agua:
            self.grafo.nodes[p]['agua'] = True
        
        self.fogo_ativo = []
        self.brigadistas_em_movimento = []

    def iniciar_fogo(self, inicio):
        if self.grafo.nodes[inicio]['brigadistas']:
            raise ValueError("O fogo não pode começar em um posto de brigadistas.")
        self.fogo_ativo = [inicio]
        self.grafo.nodes[inicio]['fogo'] = True

    def propagar_fogo(self):
        novos_focos = []
        for v in self.fogo_ativo:
            for vizinho in self.grafo.neighbors(v):
                if not self.grafo.nodes[vizinho]['fogo']:
                    novos_focos.append(vizinho)
                    self.grafo.nodes[vizinho]['fogo'] = True
        self.fogo_ativo = novos_focos

    def caminho_mais_curto(self, origem, destino):
        return nx.shortest_path(self.grafo, origem, destino, weight='weight')
    
    def deslocar_brigadistas(self):
        novos_movimentos = []
        for (origem, destino, agua_restante) in self.brigadistas_em_movimento:
            caminho = self.caminho_mais_curto(origem, destino)
            
            if len(caminho) > 1:
                novo_local = caminho[1]
                agua_restante -= 1
                
                if self.grafo.nodes[novo_local]['agua']:
                    agua_restante = self.capacidade_caminhoes
                
                if self.grafo.nodes[novo_local]['fogo'] and agua_restante > 0:
                    self.grafo.nodes[novo_local]['fogo'] = False
                    print(f"🔥 Fogo apagado em {novo_local} pelos brigadistas!")
                else:
                    novos_movimentos.append((novo_local, destino, agua_restante))
        
        self.brigadistas_em_movimento = novos_movimentos

    def enviar_brigadistas(self):
        for p in [n for n in self.grafo.nodes if self.grafo.nodes[n]['brigadistas']]:
            if self.fogo_ativo:
                destino = self.fogo_ativo[0]
                self.brigadistas_em_movimento.append((p, destino, self.capacidade_caminhoes))
                print(f"🚒 Brigadistas enviados de {p} para {destino}")
    
    def simular(self, inicio_fogo):
        self.iniciar_fogo(inicio_fogo)
        while self.fogo_ativo:
            self.propagar_fogo()
            self.enviar_brigadistas()
            self.deslocar_brigadistas()
            print(f"🔥 Fogo ativo nos vértices: {self.fogo_ativo}")
        print("✅ Simulação concluída. Todos os focos de incêndio foram extintos!")




In [2]:
# Exemplo de uso
sim = SimulacaoIncendio(num_vertices=20, num_arestas=40, postos_brigadistas=[0, 10], pontos_agua=[5, 15], capacidade_caminhoes=3)
sim.simular(inicio_fogo=7)

🚒 Brigadistas enviados de 0 para 1
🚒 Brigadistas enviados de 10 para 1
🔥 Fogo apagado em 1 pelos brigadistas!
🔥 Fogo ativo nos vértices: [1, 2, 12]
🚒 Brigadistas enviados de 0 para 5
🚒 Brigadistas enviados de 10 para 5
🔥 Fogo apagado em 5 pelos brigadistas!
🔥 Fogo apagado em 4 pelos brigadistas!
🔥 Fogo ativo nos vértices: [5, 0, 18, 11, 17, 4, 14, 6]
🚒 Brigadistas enviados de 0 para 1
🚒 Brigadistas enviados de 10 para 1
🔥 Fogo apagado em 5 pelos brigadistas!
🔥 Fogo apagado em 1 pelos brigadistas!
🔥 Fogo apagado em 4 pelos brigadistas!
🔥 Fogo ativo nos vértices: [1, 19, 13, 4, 10, 15, 3, 8, 16, 5]
🚒 Brigadistas enviados de 0 para 5
🚒 Brigadistas enviados de 10 para 5
🔥 Fogo apagado em 1 pelos brigadistas!
🔥 Fogo apagado em 4 pelos brigadistas!
🔥 Fogo ativo nos vértices: [5, 4, 9, 1]
🚒 Brigadistas enviados de 0 para 1
🚒 Brigadistas enviados de 10 para 1
🔥 Fogo apagado em 1 pelos brigadistas!
🔥 Fogo apagado em 4 pelos brigadistas!
🔥 Fogo ativo nos vértices: [1, 4]
🔥 Fogo ativo nos vértice